# Amazon SageMaker With Trained XGBoost Model

Hosting a Pre-Trained Model in Amazon SageMaker XGBoost Algorithm Container

In [1]:
%%time

import os
import boto3
import re
import json
import pandas as pd
from sagemaker import get_execution_role

CPU times: user 271 ms, sys: 45.2 ms, total: 317 ms
Wall time: 321 ms


In [2]:

endpoint_name = 'XGBoostEndpoint-2021-04-27-07-04-43'

In [3]:
sm_client = boto3.client('sagemaker')

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

runtime_client = boto3.client('sagemaker-runtime')

Status: InService


In [4]:
df = pd.read_json('input.json')
print(df)
cat_cols_few = ["loan_type", "property_type", "owner_occupancy", "agency_code", 
                "lien_status", "applicant_ethnicity", "applicant_race_1", "applicant_sex"] 

print(pd.get_dummies(df, columns = cat_cols_few))

         as_of_year  agency_code  loan_type  property_type  owner_occupancy  \
7022200        2017            3          1              1                1   

         loan_amount_000s  action_taken  msamd  state_code  county_code  ...  \
7022200               585             1  42644          53           33  ...   

         applicant_sex  applicant_income_000s  lien_status  population  \
7022200              2                    117            1        5054   

         minority_population  hud_median_family_income  tract_to_msamd_income  \
7022200            34.709999                     96000              95.730003   

         number_of_owner_occupied_units  number_of_1_to_4_family_units  \
7022200                            1499                           1675   

         tract_family_income  
7022200         91900.803223  

[1 rows x 23 columns]
         as_of_year  loan_amount_000s  action_taken  msamd  state_code  \
7022200        2017               585             1  42644  

In [5]:
y_test = df['action_taken']
X_test = df.drop(["action_taken"], axis=1)

In [6]:


file_name = 'test_point.csv' #customize to your test file, will be 'mnist.single.test' if use data above

with open(file_name, 'r') as f:
    payload = f.read().strip()

print(payload)


-2.245257779940310894e-01,-1.106398261207008460e+00,1.081772861892804860e+00,-2.970795964231225383e-01,2.943375827454213667e+00,-6.957775243550255728e-02,-4.066429056088091132e-01,-4.934460376701826911e-01,-5.206898252826737128e-01,-1.052816558185312446e-01,1.383134807327869309e-01,9.081868404232940062e-02,5.801412067275250051e-01,-4.350516079838701033e-01,-2.945254733939885461e-01,-1.140646032201983462e-01,1.594972434950011619e-01,-1.594972434950009121e-01,3.079843003489509123e-01,-3.079843003489510789e-01,-1.971527436590640170e-01,-1.758175529644079305e-01,-3.132228098906313374e-01,-3.043723169311758903e-01,9.942744436327647373e-01,-5.859700678893761117e-01,2.871108995510856521e-01,-1.986831176813227684e-01,-1.992351132870562036e-01,-3.654261994588782203e-01,3.654261994588781648e-01,-9.178574644175087338e-02,-2.715227308355318558e-01,-3.098038523218605844e-01,-7.275145742804130233e-02,4.523369375867579145e-01,6.905443954752987779e-01,-6.905443954752987779e-01,-3.288795721866425437e-0

In [7]:
%%time
response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read().decode('ascii')
response['Body'] = result
print('Predicted Class Probabilities: {}.'.format(response))

Predicted Class Probabilities: {'ResponseMetadata': {'RequestId': '6436a7a6-8194-48f2-930d-92aa695c538d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6436a7a6-8194-48f2-930d-92aa695c538d', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Thu, 29 Apr 2021 03:40:22 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '18'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': '0.0885649025440216'}.
CPU times: user 21.2 ms, sys: 107 µs, total: 21.3 ms
Wall time: 176 ms


In [8]:
file_name = 'test_y.csv' #customize to your test file, will be 'mnist.single.test' if use data above

with open(file_name, 'r') as f:
    point_y = f.read().strip()

In [9]:
def lbl(y):
    if y == 0:
        return '@ Not accepted'
    elif y==1:
        return '@ Approved'
    return 'Unkown'


predictedLabel = round(float(result),0)

print('Predicted Class Label: {}.'.format(lbl(predictedLabel)))
print('Actual Class Label: {}.'.format(lbl(float(point_y))))

Predicted Class Label: @ Not accepted.
Actual Class Label: @ Approved.
